# 무인민원발급기 위치정보

# module import

In [1]:
from bs4 import BeautifulSoup as bs 
import requests
import time 
import folium
import re
import pandas as pd
import folium

# 함수 정의

In [2]:
def getLatLng(addr):
#     addr = '서울특별시 구로구 디지털로 306'
    app_key = 'KakaoAK' + ' 78106cafe4d656639f171b177dda611f'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query': addr,
        'page': '',
        'AddressSize': ''
    }
    headers  = {
        'Authorization': app_key
    }
    resp = requests.get(url, params=params, headers=headers)
    if resp.json()['documents'] != []:
        json = resp.json()['documents'][0]
        return (json['x'], json['y'])
    else:
        fixed_addr = addr.replace(addr.split(' ')[-1], '').strip()
        return getLatLng(fixed_addr)

def get_avg_point(point_list):
    x = sum(map(lambda i:float(i[0]), point_list)) / len(point_list)
    y = sum(map(lambda i:float(i[1]), point_list)) / len(point_list)
    return y,x

def draw_map(addr):
    map_noman = folium.Map(location=get_avg_point(addr_point_list),zoom_start=7)
    disabled_sup = list(set(df_noman['장애인편의발급기형태']))
    for x, y in addr_point_list:
        if df_noman['장애인편의발급기형태'][cnt] == disabled_sup[0]:
            add_marker(map_noman, x, y)
    return map_noman

def draw_marker(x, y):
    marker = folium.Marker((y,x))
    icon=folium.Icon(color='green', icon='glyphicon glyphicon-print')
    icon.add_to(marker)
    return marker

# 원본 데이터 수신(excel -> pandas)

In [3]:
df_noman = pd.read_excel('http://www.minwon.go.kr/main?d=AA100DataDownApp&SAVE_FILE_NM=38800.xls')
df_noman_road = df_noman.sort_values(by='도로명주소')
noman_road = list(df_noman_road['도로명주소'].values)

# 사용자 데이터 입력 

In [4]:
input_addr = input('지명을 입력하세요: ')
# print(input_addr)
addr_pattern = re.compile(f'{input_addr}')
start = -1
end = -1
for i in range(len(noman_road)):
#     print(i)
#     print(noman_road[i])
    if re.search(addr_pattern, noman_road[i]):
        if start != -1:
            continue
        start = i
    else:
        if start == -1:
            continue
        end = i
        break
        
point_list = []
marker_list = []    
if len(noman_road[start:end]) <= 0:
    print('정보가 없습니다.')
elif len(noman_road[start:end]) >= 500:
    print('범위가 너무 넓습니다. 작은 범위를 입력해 주세요.')
else:
    for addr in noman_road[start:end]:
        point = getLatLng(addr)
        point_list.append(point)
        marker = draw_marker(point[0],point[1])
        marker_list.append(marker)


지명을 입력하세요: 안산


In [14]:
max_point = max(map(lambda i:float(i[0]), point_list))
min_point = min(map(lambda i:float(i[0]), point_list))
max_point - min_point

0.32773326825599725

# 정보 시각화

In [67]:
map_noman = folium.Map(location=get_avg_point(point_list), zoom_start=12)
for marker in marker_list:
    marker.add_to(map_noman)
map_noman

# 개선사항
## 지명 입력을 제대로 받기가 힘듦